In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats

#Introduction

The success of a new shoe model depends on multiple factors, including style, comfort, stability, cushioning, and durability. To ensure a high-quality product, a shoe company has established specific benchmark values for each of these criteria. The latest prototype, Model 1, has been evaluated by 25 individuals, and their ratings have been recorded.



This report aims to analyze the collected data and assess whether Model 1 meets the company's expected standards. The evaluation involves calculating the average ratings for each criterion, comparing them against the predefined goals, and determining if the shoe is suitable for market release. Based on the findings, a final recommendation will be provided regarding the readiness of Model 1 for commercial launch.

In [ ]:
df = pd.read_excel("Book2.xlsx")
df.head()

,Subject,Style,Comfort,Stability,Cushion,Durability
0,1,8,5,10,2,19
1,2,4,5,3,4,9
2,3,1,7,7,6,14
3,4,7,4,5,0,10
4,5,6,1,9,1,0


In [ ]:
df.shape

(25, 6)

# One-sample Hotelling $T^2$ statistic <br>



Expected goals for each criterion:<br>
- Style	: 7
- Comfort	: 8
- Stability	: 5
- Cushion : 7
- Durability : 9 <br>



In [ ]:
df.drop(columns="Subject",inplace=True)

## 1) Hypothesis

$H_0$ - Null Hypothesis- The average ratings for Model 1 meet the company's goal standards. (μ = $μ_0$)<br>

$H_1$ - Alternative Hypothesis- The average ratings for Model 1 do not meet the company's goal standards.(μ $\neq$ $μ_0$)

## 2) Assumptions and Inferences:
#### Statistical Assumptions
- The observations (product attributes like Comfort, Stability, etc.) are normally distributed.
- The samples are independent (each rating is unbiased and not influenced by others).
- The covariance structure is valid (ensuring no extreme collinearity between variables).
- The hypothesized mean vector **𝜇₀** represents an expected goal standard by the company

#### Business Assumptions
- If the mean significantly differs, we may need product redesign or improvement.
- If the mean is not significantly different, we may continue with the existing model.

In [ ]:
df_compare=pd.DataFrame(df.mean(),columns=["sample_mean"])
df_compare

,sample_mean
Style,4.88
Comfort,4.92
Stability,6.00
Cushion,5.40
Durability,10.60


In [ ]:
df_compare["mu_0"]=[7,8,5,7,9]
df_compare

,sample_mean,mu_0
Style,4.88,7
Comfort,4.92,8
Stability,6.00,5
Cushion,5.40,7
Durability,10.60,9


In [ ]:
df_std_dev=df.std()
df_std_dev

,0
Style,3.218695
Comfort,3.094619
Stability,2.872281
Cushion,3.253204
Durability,6.370505


In [ ]:
S=df.cov()
S

,Style,Comfort,Stability,Cushion,Durability
Style,10.360000,1.031667,0.250000,-0.491667,4.825000
Comfort,1.031667,9.576667,-1.000000,-0.216667,6.925000
Stability,0.250000,-1.000000,8.250000,3.958333,-5.708333
Cushion,-0.491667,-0.216667,3.958333,10.583333,-3.166667
Durability,4.825000,6.925000,-5.708333,-3.166667,40.583333


In [ ]:
S_inv=np.linalg.inv(S)
S_inv

array([[ 0.10385511, -0.00288689, -0.01613663,  0.00673194, -0.01359929],
       [-0.00288689,  0.11938471,  0.0030425 , -0.00480477, -0.01997513],
       [-0.01613663,  0.0030425 ,  0.16248262, -0.05549712,  0.0199233 ],
       [ 0.00673194, -0.00480477, -0.05549712,  0.11583392,  0.00125181],
       [-0.01359929, -0.01997513,  0.0199233 ,  0.00125181,  0.03256601]])

In [ ]:
df_compare["deviation"]=df_compare["sample_mean"]-df_compare["mu_0"]
df_compare

,sample_mean,mu_0,deviation
Style,4.88,7,-2.12
Comfort,4.92,8,-3.08
Stability,6.00,5,1.00
Cushion,5.40,7,-1.60
Durability,10.60,9,1.60


In [ ]:
# number of variables
p=df.shape[1]
#number of observations
n=df.shape[0]
p, n

(5, 25)

In [ ]:
def Hotelling_Tsq(deviation,n,cov_inv):
  return (deviation.T@cov_inv@deviation)*n

In [ ]:
T_sq=Hotelling_Tsq(df_compare["deviation"],n,S_inv)
T_sq

66.90107950191121

In [ ]:
#F Statistic

F = ((n - p)/(p*(n-1)))*T_sq
F

11.150179916985202

In [ ]:
from scipy.stats import f

alpha = 0.05 #95% confidence
dofn = p
dofd = n-p

f_crit = f.ppf(1 - alpha, dofn, dofd)
f_crit

2.7108898372096917

In [ ]:
from scipy import stats
p_value = 1 - stats.f.cdf(F, p, n+p)
p_value

3.908169041277887e-06

In [ ]:

print("Sample Means:", df_compare["sample_mean"].values)
print("\nVariance-Covariance Matrix:\n", S)
print("\nF Calculated:", F)
print("\nF Critical:", f_crit)
print("\nP-value:", p_value)

print("\nInferences:")
if p_value < alpha:
    print("Reject the null hypothesis.")
    print("The average ratings for Model 1 do not meet the company's goal standards.")
else:
    print("Fail to reject the null hypothesis.")
    print("There is not enough evidence to suggest that the average ratings for Model 1 do not meet the company's goal standards.")


Sample Means: [ 4.88  4.92  6.    5.4  10.6 ]

Variance-Covariance Matrix:
                 Style   Comfort  Stability    Cushion  Durability
Style       10.360000  1.031667   0.250000  -0.491667    4.825000
Comfort      1.031667  9.576667  -1.000000  -0.216667    6.925000
Stability    0.250000 -1.000000   8.250000   3.958333   -5.708333
Cushion     -0.491667 -0.216667   3.958333  10.583333   -3.166667
Durability   4.825000  6.925000  -5.708333  -3.166667   40.583333

F Calculated: 11.150179916985202

F Critical: 2.7108898372096917

P-value: 3.908169041277887e-06

Inferences:
Reject the null hypothesis.
The average ratings for Model 1 do not meet the company's goal standards.


##Confidence Intervals

In [ ]:
ci = []
for i in range(0, len(df_compare["sample_mean"])):
  critical_z = stats.norm.ppf(1 - alpha / 2)
  margin_of_error = critical_z*(df_std_dev.iloc[i]/np.sqrt(n))
  confidence_interval = ((df_compare["sample_mean"].iloc[i] - margin_of_error).round(2), (df_compare["sample_mean"].iloc[i] + margin_of_error).round(2))
  ci.append(confidence_interval)
ci

[(3.62, 6.14), (3.71, 6.13), (4.87, 7.13), (4.12, 6.68), (8.1, 13.1)]

In [ ]:
import numpy as np
from scipy import stats

def calc_ci(df_compare, df_std_dev, n, alpha=0.05):
    ci = []

    for i in range(len(df_compare["sample_mean"])):
        critical_z = stats.norm.ppf(1 - alpha / 2)  # Z-critical value for confidence interval
        margin_of_error = critical_z * (df_std_dev.iloc[i] / np.sqrt(n))

        confidence_interval = (
            round(df_compare["sample_mean"].iloc[i] - margin_of_error, 2),
            round(df_compare["sample_mean"].iloc[i] + margin_of_error, 2)
        )
        ci.append(confidence_interval)

        print(f'CI of {df_compare.index[i]} w/ Sample mean of {df_compare["sample_mean"].iloc[i]}:')
        print(f'  UCL -> {confidence_interval[1]}, LCL -> {confidence_interval[0]}')

    print("\nInference based on CI:")
    for i in range(len(df_compare["sample_mean"])):
        if ci[i][0] > df_compare["mu_0"].iloc[i]:
            print(f"Average {df_compare.index[i]} rating is MORE than the goal standard level!")
        elif ci[i][1] < df_compare["mu_0"].iloc[i]:
            print(f"Average {df_compare.index[i]} rating is LESS than the goal standard level!")
        else:
            print(f"Average {df_compare.index[i]} rating is within the expected range.")

    return ci

# Example usage:
ci = calc_ci(df_compare, df_std_dev, n)


CI of Style w/ Sample mean of 4.88:
  UCL -> 6.14, LCL -> 3.62
CI of Comfort w/ Sample mean of 4.92:
  UCL -> 6.13, LCL -> 3.71
CI of Stability w/ Sample mean of 6.0:
  UCL -> 7.13, LCL -> 4.87
CI of Cushion w/ Sample mean of 5.4:
  UCL -> 6.68, LCL -> 4.12
CI of Durability w/ Sample mean of 10.6:
  UCL -> 13.1, LCL -> 8.1

Inference based on CI:
Average Style rating is LESS than the goal standard level!
Average Comfort rating is LESS than the goal standard level!
Average Stability rating is within the expected range.
Average Cushion rating is LESS than the goal standard level!
Average Durability rating is within the expected range.


In [ ]:
def lengths(ci):
  lengths = []
  for i in range(0, len(df_compare["sample_mean"])):
    length = ci[i][1] - ci[i][0]
    lengths.append(length.round(1))
  return lengths

lengths_ci = lengths(ci)
lengths_ci

[2.5, 2.4, 2.3, 2.6, 5.0]

In [ ]:
c2 = (p*(n-1)/(n-p))*f.ppf(1 - 0.05, dofn, dofd)
c2

16.26533902325815

##Simultaneous Confidence Interval

In [ ]:

means = np.array(df_compare["sample_mean"])
std_dev = np.array(df_std_dev)

sci = []
for i in range(0, len(means)):

  margin_of_error =np.sqrt((c2*(std_dev[i]**2))/n)

  s_confidence_interval = ((means[i] - margin_of_error).round(1), (means[i] + margin_of_error).round(1))
  sci.append(s_confidence_interval)
sci

[(2.3, 7.5), (2.4, 7.4), (3.7, 8.3), (2.8, 8.0), (5.5, 15.7)]

In [ ]:

def sci_lengths(s_ci):
  sci_lengths = []
  for i in range(0, len(means)):

    s_length =sci[i][1]-sci[i][0]

    sci_lengths.append(s_length.round(2))
  return sci_lengths

lengths_sci = sci_lengths(sci)
lengths_sci

[5.2, 5.0, 4.6, 5.2, 10.2]

In [ ]:
for i in range(0, len(means)):
  print(f'Simultaneous CI of {df.columns[i]} w/ Sample mean of {means[i]} is: UCL -> {sci[i][1]}, LCL -> {sci[i][0]}')

print(f'\n')

for i in range(0, len(means)):
  if sci[i][0] > df_compare["mu_0"].iloc[i]:
    print(f"Average {df.columns[i]} rating is MORE than the goal standard level!") #because both interval limits are greater than Recommended intake")
  elif sci[i][1] < df_compare["mu_0"].iloc[i]:
    print(f"Average {df.columns[i]} rating is LESS than the goal standard level!") #because both interval limits are lesser than Recommended intake")


Simultaneous CI of Style w/ Sample mean of 4.88 is: UCL -> 7.5, LCL -> 2.3
Simultaneous CI of Comfort w/ Sample mean of 4.92 is: UCL -> 7.4, LCL -> 2.4
Simultaneous CI of Stability w/ Sample mean of 6.0 is: UCL -> 8.3, LCL -> 3.7
Simultaneous CI of Cushion w/ Sample mean of 5.4 is: UCL -> 8.0, LCL -> 2.8
Simultaneous CI of Durability w/ Sample mean of 10.6 is: UCL -> 15.7, LCL -> 5.5


Average Comfort rating is LESS than the goal standard level!


In [ ]:
print("\nBusiness Recommendation:")

for i in range(len(means)):
    criterion = df.columns[i]
    lower_bound = sci[i][0]
    upper_bound = sci[i][1]
    goal = df_compare["mu_0"].iloc[i]

    if lower_bound > goal :
        print(f"- {criterion}: The average rating ({means[i]:.2f}) significantly exceeds the goal ({goal}).  No action needed.")
    elif upper_bound < goal:
        print(f"- {criterion}: The average rating ({means[i]:.2f}) falls significantly below the goal ({goal}). Investigate and consider product redesign or improvement.")
    else:
        print(f"- {criterion}: The average rating ({means[i]:.2f}) is within an acceptable range of the goal ({goal}). Monitor performance.")

if all(lower_bound > goal for lower_bound, goal in zip([x[0] for x in sci], df_compare["mu_0"])):
    print("\nOverall: Model 1 meets all quality standards and is ready for market release.")
else:
    print("\nOverall: Model 1 requires further review and potential modifications before market release.")



Business Recommendation:
- Style: The average rating (4.88) is within an acceptable range of the goal (7). Monitor performance.
- Comfort: The average rating (4.92) falls significantly below the goal (8). Investigate and consider product redesign or improvement.
- Stability: The average rating (6.00) is within an acceptable range of the goal (5). Monitor performance.
- Cushion: The average rating (5.40) is within an acceptable range of the goal (7). Monitor performance.
- Durability: The average rating (10.60) is within an acceptable range of the goal (9). Monitor performance.

Overall: Model 1 requires further review and potential modifications before market release.


## Business decisions :


Yes, the results indicate a clear issue with **Comfort**, **Cushion** and **Style**. The business decisions should focus on:

- **Product Design Improvements** - Addressing Comfort and Style ratings.
- **Market Research** - Understanding why customers rate these factors low.
- The confidence interval for Comfort (CI: (3.71, 6.13)) is entirely below the expected value of mu_0 = 8.
- This means that even in the best-case scenario, Comfort does not reach the goal standard.
- Action: Improve material design, cushioning, or ergonomic features to enhance Comfort.

From our analysis, we observe that the simultaneous confidence intervals tend to be wider than the univariate confidence intervals. This is because simultaneous CIs account for all possible linear combinations of variables, ensuring a more conservative estimate to control the overall error rate.